![Image](./snapshot_strategy.png)

1차에서 수행한 100개의 질문을 제외하고 다시 랜덤하게 100개 추출


In [1]:
file_pre = '4th'

In [2]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
# p = p+r'\config'
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [3]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se
import lib.annotation.D_Annotation as da
import lib.annotation.Self_Consistency as sc
import re
import datetime
import pandas as pd
import re
import numpy as np
from sklearn import metrics


INFO 11-30 15:13:32 [__init__.py:216] Automatically detected platform cuda.


In [4]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [5]:
q_sql_2223 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-12-01' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from public.tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [6]:
# 2324의 경우 public_for_2324 스키마 tt_posts_difficulty_annotated테이블을 보고 제외
q_sql_2324 = """select a.id 
                , a.creationdate
                , a.title
                , b.body
            from public_for_2324.posts a 
               , public_for_2324.postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2024-12-03' 
              and a.posttypeid = '1'
              and a.tags like '%<python>%' 
              and not exists(select 1 
                                    from public_for_2324.tt_posts_difficulty_annotated x 
                                where a.id = x.id)
              and not exists(select 1 
                                    from tt_posts_difficulty_done xx 
                                where a.id = xx.id)
"""

In [7]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2223)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [8]:
q_output_for_2223 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [9]:
q_output_for_2223['22_yn'] = np.where(pd.to_datetime(q_output_for_2223['creationdate']).dt.date < datetime.date(2022, 12, 1), 1, 0)

q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

In [10]:
qid_for_2223_22 = np.random.choice(list(q_output_for_2223_22['id']), size=54, replace=False)
qid_for_2223_23 = np.random.choice(list(q_output_for_2223_23['id']), size=54, replace=False)

In [11]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_2324)
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [12]:
q_output_for_2324 = pd.DataFrame(rows, columns = [
  'id'
  ,'creationdate'
  ,'title'
  , 'body'
])


In [13]:
q_output_for_2324['22_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2022, 12, 1), 1, 0)
q_output_for_2324['23_yn'] = np.where((pd.to_datetime(q_output_for_2324['creationdate']).dt.date < datetime.date(2023, 12, 2)) & (q_output_for_2324['22_yn'] ==0), 1, 0)
q_output_for_2324['24_yn'] = np.where(pd.to_datetime(q_output_for_2324['creationdate']).dt.date >= datetime.date(2023, 12, 2), 1, 0)

print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['22_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['23_yn'] ==1, 'creationdate'].max())

print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].min())
print(q_output_for_2324.loc[q_output_for_2324['24_yn'] ==1, 'creationdate'].max())

# q_output_for_2223_22 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 1, :]
# q_output_for_2223_23 = q_output_for_2223.loc[q_output_for_2223['22_yn'] == 0, :]
# first_ann_q_id = np.random.choice(list(q_output['id']), size=30, replace=False)

2021-11-30 00:07:54.443000
2022-11-30 23:58:57.893000
2022-12-01 00:01:48.020000
2023-12-01 23:31:23.847000
2023-12-02 00:14:18.247000
2024-12-02 23:51:31.780000


In [14]:
q_output_for_2324_22 = q_output_for_2324.loc[q_output_for_2324['22_yn'] == 1, :]
q_output_for_2324_23 = q_output_for_2324.loc[q_output_for_2324['23_yn'] == 1, :]
q_output_for_2324_24 = q_output_for_2324.loc[q_output_for_2324['24_yn'] == 1, :]

In [15]:
print(q_output_for_2324_22['creationdate'].min())
print(q_output_for_2324_22['creationdate'].max())

print(q_output_for_2324_23['creationdate'].min())
print(q_output_for_2324_23['creationdate'].max())

print(q_output_for_2324_24['creationdate'].min())
print(q_output_for_2324_24['creationdate'].max())

2021-11-30 00:07:54.443000
2022-11-30 23:58:57.893000
2022-12-01 00:01:48.020000
2023-12-01 23:31:23.847000
2023-12-02 00:14:18.247000
2024-12-02 23:51:31.780000


In [16]:
qid_for_2324_22 = np.random.choice(list(q_output_for_2324_22['id']), size=36, replace=False)
qid_for_2324_23 = np.random.choice(list(q_output_for_2324_23['id']), size=36, replace=False)
qid_for_2324_24 = np.random.choice(list(q_output_for_2324_24['id']), size=36, replace=False)

In [17]:
def chg_tag(code):
    st_pattern = r'<pre(?: class="[^"]*")?><code>'
    st_dst = "```python\n"
    code = re.sub(st_pattern, st_dst, code, count=0, flags=0)
    
    end_dst = "```"
    end_pattern =r'</code></pre>'
    code = re.sub(end_pattern, end_dst, code, count=0, flags=0)
    return code
                                                

In [18]:
def clean_code_df(q_output, q_list):
    q_output_copy = q_output[q_output['id'].isin(q_list)].copy()
    q_output_copy['t_body'] = q_output_copy['body'].apply(chg_tag)
    q_output_copy['clean_body'] = q_output_copy['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))
    q_output_copy['clean_body'] = q_output_copy['clean_body'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
    q_output_copy['question'] = """<Title>"""+q_output_copy['title'].map(str)+"""</Title>. <Question>"""+q_output_copy['clean_body'].map(str)+"""</Question> Let's think through the difficulty of question carefully, step by step. """
    return q_output_copy


In [19]:
pp_2324_22 = clean_code_df(q_output_for_2223_22, qid_for_2223_22)
pp_2324_22 = clean_code_df(q_output_for_2223_23, qid_for_2223_23)

pp_2324_22 = clean_code_df(q_output_for_2324_22, qid_for_2324_22)
pp_2324_23 = clean_code_df(q_output_for_2324_23, qid_for_2324_23)
pp_2324_24 = clean_code_df(q_output_for_2324_24, qid_for_2324_24)

In [20]:
snapshop1_sample = pd.concat((pp_2324_22[['id', 'question']], pp_2324_22[['id', 'question']]))
snapshop2_sample = pd.concat((pp_2324_22[['id', 'question']], pp_2324_23[['id', 'question']], pp_2324_24[['id', 'question']]))

In [21]:
snapshop1_sample[['id', 'question']].to_csv(f'./{file_pre}_snapshop1_sample.csv')
snapshop2_sample[['id', 'question']].to_csv(f'./{file_pre}_snapshop2_sample.csv')

## public_for_2324.tt_posts_difficulty_annotated, public.tt_posts_difficulty_annotated 테이블에 인서트 필요

In [22]:
list(snapshop1_sample['id'])

[70264389,
 70274885,
 70313318,
 70540832,
 70693775,
 71142590,
 71257381,
 71642549,
 72031953,
 72411313,
 72942199,
 73078635,
 73207452,
 73323356,
 73418818,
 73663545,
 73691847,
 73742819,
 70597183,
 70770799,
 71086503,
 71461507,
 72242042,
 72592491,
 73052949,
 72805067,
 72889403,
 73639509,
 73280069,
 73325981,
 73937823,
 73880466,
 74020058,
 74319305,
 74552305,
 74075528,
 70264389,
 70274885,
 70313318,
 70540832,
 70693775,
 71142590,
 71257381,
 71642549,
 72031953,
 72411313,
 72942199,
 73078635,
 73207452,
 73323356,
 73418818,
 73663545,
 73691847,
 73742819,
 70597183,
 70770799,
 71086503,
 71461507,
 72242042,
 72592491,
 73052949,
 72805067,
 72889403,
 73639509,
 73280069,
 73325981,
 73937823,
 73880466,
 74020058,
 74319305,
 74552305,
 74075528]

In [23]:
scn_smp = pd.read_csv(f'./2nd_snapshop2_sample.csv')
thd_smp = pd.read_csv(f'./3rd_snapshop2_sample.csv')
tar_smp = pd.read_csv(f'/usr/share/d_ollama/data/q_output_code_y_74_bak.csv')

In [24]:
sample_smp = pd.concat([scn_smp, thd_smp], axis = 0)

In [25]:
sample_smp.tail()

,Unnamed: 0,id,question
103,337884,78644705,<Title>How to add a min/max value in condition...
104,341130,78877362,<Title>Python webscraping for websockets</Titl...
105,343934,79027269,<Title>can i run python codebase with endpint ...
106,346308,79188432,<Title>Clickable text in ScrolledText widget -...
107,347807,79243311,<Title>Unable to get changes in General Model ...


In [26]:
tot_df = pd.merge(tar_smp, sample_smp[['id', 'question']], on = 'id', how='left')

In [27]:
mapping = {'Basic': 'Difficulty Level : Basic', 
           'Intermediate': 'Difficulty Level : Intermediate', 
           'Advanced' : 'Difficulty Level : Advanced'}
tot_df['answer'] = tot_df['answer'].map(mapping)

In [28]:
tot_df.to_csv('/usr/share/d_ollama/data/q_output_code_y_74.csv')

In [29]:
import os
os.environ["VLLM_LOG_LEVEL"] = "ERROR"

In [30]:
from multiprocessing import Process
    
def task(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temperature, excel_ver):

    print(f"Task {llm_model}_{few_shot_n}_{test_n}_{q_src_yn}_{p_ver}_{sc_num} 시작")
    for i in range(ver):
        print(f"Task {llm_model}|{few_shot_n}|{test_n}|{q_src_yn}|{ver}|{p_ver}|{sc_num}|{temperature}|{excel_ver} 실행 중: {i}")

        sc.Self_Consistency(   llm_model
                            , few_shot_n
                            , test_n
                            , q_src_yn
                            , ver
                            , p_ver
                            , sc_num
                            , temperature
                            , excel_ver
                            , i)
    
    print(f"Task {llm_model}_{few_shot_n}_{test_n}_{q_src_yn}_{p_ver}_{sc_num} 완료")

if __name__ == "__main__":


    process3 = Process(target=task, args=('v', 4, 30, 'Y', 100, 'sys_prompt10', 5, 0.01, 'ver6'))
    process3.start()
    
    process3.join()
    print("모든 작업 완료")


Task v_4_30_Y_sys_prompt10_5 시작
Task v|4|30|Y|100|sys_prompt10|5|0.01|ver6 실행 중: 0
./result/sc_v_result_4_30_Y_100_sys_prompt10_5_0.01_ver6_0.csv
/usr/share/d_ollama/data/q_output_code_y_74.csv
VLLM


Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1098051/2533440307.py", line 9, in task
    sc.Self_Consistency(   llm_model
  File "/home/mghan/sopjt/git/stackoverflow_src_2425/lib/annotation/Self_Consistency.py", line 29, in __init__
    self.calc_acc(llm_model, few_shot_n, q_src_yn)
  File "/home/mghan/sopjt/git/stackoverflow_src_2425/lib/annotation/Self_Consistency.py", line 168, in calc_acc
    self.calc_acc_for_v(llm_model, few_shot_n, q_src_yn)
  File "/home/mghan/sopjt/git/stackoverflow_src_2425/lib/annotation/Self_Consistency.py", line 110, in calc_acc_for_v
    vllm = VLLM()
  File "/home/mghan/sopjt/git/stackoverflow_src_2425/lib/annotation/VLLM.py", line 16, in __init__
    self.llm = LLM( model="/usr/share/d_ollama/.ollama/models/hf

모든 작업 완료


In [31]:
path = './result'
file_list = os.listdir(path)

In [32]:
def sc_calc_acc_condition_with_temp_with_sc(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temp, excel_ver):
    tmp = pd.DataFrame()
    df_eval = pd.DataFrame()
    acc_list = []
    path = './result'
    # ./result/sc_l_result_4_15_Y_30_sys_prompt8_0.01_ver1_0.csv
    file_list = os.listdir(path)
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')]
    opt_file = [x for x in opt_file if x.endswith(f'.csv')]

    df = pd.DataFrame()

    
    if len(opt_file)>0 : 
        for f in opt_file:
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp = tmp.dropna()

            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp = tmp[tmp['o_result'].isin(['1', '0', '2'])]

            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            acc_list.append(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        y_true = df['o_result']
        y_pred = df['gold']
        print(metrics.classification_report(y_true, y_pred, digits=3))
        
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)
        return acc_list


In [33]:
# Process(target=task, args=('l', 1, 10, 'Y', 10, 'sys_prompt10', 3, 0.01, 'ver5'))
# Process(target=task, args=('l', 2, 10, 'Y', 10, 'sys_prompt10', 3, 0.01, 'ver5'))
# Process(target=task, args=('l', 4, 10, 'Y', 10, 'sys_prompt10', 5, 0.01, 'ver5'))

In [34]:
# process3 = Process(target=task, args=                 Process(target=task, args=('v', 4, 5, 'Y', 20, 'sys_prompt10', 5, 0.01, 'ver6'))
list_ =         sc_calc_acc_condition_with_temp_with_sc('v', 4, 5, 'Y', 20, 'sys_prompt10', 5,  0.01, 'ver6')
print(list_)

              precision    recall  f1-score   support

           0      1.000     0.714     0.833        21
           1      0.708     0.944     0.810        18
           2      0.909     0.909     0.909        11

    accuracy                          0.840        50
   macro avg      0.872     0.856     0.851        50
weighted avg      0.875     0.840     0.841        50

v_result_4_5_Y :  84.0
[np.float64(100.0), np.float64(100.0), np.float64(100.0), np.float64(66.66666666666666), np.float64(80.0), np.float64(100.0), np.float64(100.0), np.float64(100.0), np.float64(75.0), np.float64(66.66666666666666), np.float64(100.0), np.float64(100.0), np.float64(100.0), np.float64(100.0), np.float64(75.0), np.float64(100.0), np.float64(50.0), np.float64(0.0), np.float64(100.0), np.float64(50.0)]
